In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow
from  keras.models import Sequential, Model
from  keras.layers import Input, Dense, Add,LeakyReLU, Activation, Concatenate,Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, InputLayer, Flatten, BatchNormalization, Reshape, Lambda

from  keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau,Callback
#from  keras.applications.vgg16 import VGG16, preprocess_input
#from  keras.applications import ResNet152V2,ResNet101V2,ResNet50V2 
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input 

from  keras.optimizers import RMSprop, Adam, Adadelta
from  keras.preprocessing.image import load_img, ImageDataGenerator
from  keras.utils import multi_gpu_model
import keras.regularizers as regularizers
from tensorflow.keras.backend import epsilon
from IPython.display import Image 

import keras_metrics
import pandas
import ast
import numpy as np
import matplotlib.patches as patches 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageDraw,ImageFont

Using TensorFlow backend.


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras.backend as K
import numpy as np
import random

In [4]:
BATCH_SIZE = 32
IMG_SIZE = 224

In [5]:
def YOLF_LOSS(y_true, y_pred):
    #tensorflow.print("tensors:", y_pred)
    #y_pred = tensorflow.reshape(y_pred, [tensorflow.shape(y_pred)[0], 7, 7, 5])
    #batch_size = y_true.shape[0]
    b_p_pred = y_pred[:,0]
    b_x_pred = y_pred[:,1]
    b_y_pred = y_pred[:,2]
    b_w_pred = y_pred[:,3]
    b_h_pred = y_pred[:,4]

    b_p = y_true[:,0]
    b_x = y_true[:,1]
    b_y = y_true[:,2]
    b_w = y_true[:,3]
    b_h = y_true[:,4]
    loss_xy = np.sum(b_p * (np.square(b_x - (b_x_pred)) + np.square(b_y - (b_y_pred))))
    #b_w = np.sqrt(b_w)
    #b_h = np.sqrt(b_h)
    #b_w_pred = np.sqrt(b_w_pred)
    #b_h_pred = np.sqrt(b_h_pred)
    loss_wh = np.sum(
        b_p * 
        (
            (np.square(b_w - b_w_pred)) + 
            (np.square(b_h - b_h_pred))
        ))
    return (loss_wh+loss_xy)/BATCH_SIZE

import numpy as np
import tensorflow as tf

# create two inputs simulating a batch_size = 3
# shape (3,5)
y_true = np.array([[1,5,2,3,0.5], [1,5,2,3,0.5], [1,5,2,3,0.5]])
y_pred = np.array([[2,4,3,3,0.7], [2,4,3,3,0.7], [2,4,3,3,0.7]])

# call the first function
result = IOU(y_true, y_pred)

print(result)
print(type(result))

In [6]:
train_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/dataset_face_detection/train.csv")
valid_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/dataset_face_detection/val.csv")
test_df = pandas.read_csv("/data01/ML/dataset/FACE_CLASSIFIER/dataset_face_detection/test.csv")

In [7]:
train_df.head(3)

,image_path,x,y,x_2,y_2
0,/data01/ML/dataset/FACE_CLASSIFIER/CelebA2/Cel...,54,39,125,127
1,/data01/ML/dataset/FACE_CLASSIFIER/CelebA2/Cel...,59,20,88,106
2,/data01/ML/dataset/FACE_CLASSIFIER/CelebA2/Cel...,94,48,45,90


def convert_paths(path_string):
    temp = path_string.replace("\\", "/")  
    return "/data01/ML/" + temp.split("/",1)[1]

train_df["image_path"] = train_df["image_path"].apply(convert_paths)
valid_df["image_path"] = valid_df["image_path"].apply(convert_paths)
test_df["image_path"] = test_df["image_path"].apply(convert_paths)

In [8]:
import numpy as np

def rgb_to_gray(img_temp):
    img_temp = np.dot(img_temp[...,:3], [0.2989, 0.5870, 0.1140])
    img_temp = np.reshape(img_temp, (224,224,1))
    img_temp = np.repeat(img_temp, 3, 2)
    img_temp = preprocess_input(img_temp)
    return img_temp

In [9]:
train_datagen = ImageDataGenerator(
    #rescale = .1/255,
    #samplewise_center = True,
    #samplewise_std_normalization = True,
    preprocessing_function = rgb_to_gray)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory=None,
        x_col="image_path",
        y_col=["x","y","x_2","y_2"],
        class_mode="raw",
        shuffle=True,
        color_mode = 'rgb',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

valid_generator = train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        directory=None,
        x_col="image_path",
        y_col=["x","y","x_2","y_2"],
        class_mode="raw",
        shuffle=True,
        color_mode = 'rgb',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

Found 50575 validated image filenames.
Found 8925 validated image filenames.


In [10]:
count = 0 
for el in train_generator: 
    count = count +1 
    #print(el[1]*0.5)
    #print(IOU(el[1],el[1]*0.5))
    image = el[0][0]
    #print(image)
    #print(image.shape)
    classf = el[1][0]
    #print(classf)
    img_pil = Image.fromarray((image).astype(np.uint8))
    img1 = ImageDraw.Draw(img_pil)
    #img1.rectangle([(classf[0],classf[1]),(classf[0]+classf[2],classf[1]+classf[3])], outline ="red")
    img1.rectangle([(classf[0],classf[1]),(classf[0]+classf[2],classf[1]+classf[3])], outline ="red")
    #img_pil.show()
    img_pil.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/results/output"+ str(count) + ".png")
    #img_test = np.reshape(image, (224,224))
    #print(img.shape)
    #plt.imshow(image.astype(np.uint8))
    #plt.imshow(img_test.astype(np.uint8), cmap='Greys_r',vmin=0, vmax=255)
    #plt.show()
    if count==30:
        break

In [11]:
step_size_train = train_generator.samples/train_generator.batch_size
step_size_valid = valid_generator.samples/valid_generator.batch_size

vgg_face_classifier = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3))
vgg_face_classifier.trainable = True

#model = vgg_face_classifier.layers[-1].output

#inp = Input(shape=(IMG_SIZE,IMG_SIZE,3))
#model = vgg_face_classifier.layers[1](inp)
#model = (BatchNormalization())(model)
#for layer in vgg_face_classifier.layers[2:]:
    #model = layer(model)
        #if "_conv" in layer.name or "_pool" in layer.name:
        #model = (BatchNormalization())(model)
        #detector = (Activation("relu"))(detector)
    #print(layer)
    #if "pool" in layer.name:
        #model = BatchNormalization()(model)

In [12]:
model =  MobileNetV2(include_top=False, weights='imagenet', input_shape=(224,224,3), alpha = 1.4)
#model.trainable = True
#model.summary()

window = 4
#print(model.layers)
for layer in vgg.layers[:len(vgg.layers)-window]:
    layer.trainable = False 
vgg.summary()

In [13]:
x = model.layers[-1].output
x = Conv2D(1792,7, strides=(1,1), padding ="same", name="detector_conv_1", activation = "relu")(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Conv2D(4,7, strides=(1,1), name="detector_conv_2", activation = "relu")(x)
x = Reshape((4,), name = "coords")(x)
model = Model(input = [model.input], outputs = x)
print(len(model.layers))

160


In [14]:
for layer in model.layers[:len(model.layers)-10]:
    layer.trainable = False

In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 48) 1296        Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 48) 192         Conv1[0][0]                      
____________________________________________________________________________________________

In [16]:
#model = Concatenate()([classifier, detector])

In [17]:
#model = Model(inputs=[inp], outputs=[classifier,detector])

# Training

In [18]:
model = multi_gpu_model(model,gpus=2)

In [19]:
#model.load_weights('vgg_face_detector_YOLF_MODIFICATA.h5')

In [20]:
class Validation(Callback):
    def __init__(self, generator):
        self.generator = generator

    def on_epoch_end(self, epoch, logs):
        mse = 0
        intersections = 0
        unions = 0

        for i in range(len(self.generator)):
            batch_images, gt = self.generator[i]
            #print(gt[0])
            #print(self.generator[i])
            pred = self.model.predict_on_batch(batch_images)
            #print(pred[0])
            #mse += np.linalg.norm(gt - pred, ord='fro') / pred.shape[0]

            pred = np.maximum(pred, 0)
            diff_width = np.minimum(gt[:,0] + gt[:,2], pred[:,0] + pred[:,2]) - np.maximum(gt[:,0], pred[:,0])
            diff_height = np.minimum(gt[:,1] + gt[:,3], pred[:,1] + pred[:,3]) - np.maximum(gt[:,1], pred[:,1])
            intersection = np.maximum(diff_width, 0) * np.maximum(diff_height, 0)
            #print("inter",intersection[0])
            area_gt = gt[:,2] * gt[:,3]
            #print("area 1",area_gt[0])
            area_pred = pred[:,2] * pred[:,3]
            #print("area 2",area_pred[0])
            union = np.maximum(area_gt + area_pred - intersection, 0)
            #print("union",union[0])

            intersections += np.sum(intersection * (union > 0))
            unions += np.sum(union)

        iou = np.round(intersections / (unions + epsilon()), 4)
        logs["val_iou"] = iou

        #mse = np.round(mse, 4)
        #logs["val_mse"] = mse
        print("val_iou: {}".format(iou))
        #print(" - val_iou: {} - val_mse: {}".format(iou, mse))

training_tuples = [(200,1e-4),(30,1e-6),(20,1e-8)]
for tupl in training_tuples:
    print(tupl[0],tupl[1])

In [ ]:
#earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
#model.compile(optimizer=Adam(lr = tupl[1]), loss={"class":"binary_crossentropy","coords":"mse"}, metrics=[])
validation_datagen = Validation(generator=valid_generator)
model.compile(optimizer=Adam(lr = 1e-3), loss="mse", metrics=[])
mcp_save = ModelCheckpoint("mobile_net_test.h5", monitor="val_iou", verbose=1, save_best_only=True, save_weights_only=True, mode="max")
#mcp_save = ModelCheckpoint('vgg_face_detector_v2.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr = 1e-7, verbose=1, mode='min')
history = model.fit_generator(generator=train_generator, epochs=100, steps_per_epoch=step_size_train, validation_data=valid_generator, validation_steps=step_size_valid, verbose=1, shuffle = True, callbacks=[validation_datagen,mcp_save,reduce_lr_loss])


Epoch 1/100
1020/1580 [==================>...........] - ETA: 6:10 - loss: 242.2194

# Testing

In [ ]:
test_datagen = ImageDataGenerator(#samplewise_center = True,
    #samplewise_std_normalization = True,
    preprocessing_function = preprocess_input
)

test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory=None,
        x_col="image_path",
        y_col=["x","y","x_2","y_2"],
        class_mode="raw",
        shuffle=False,
        color_mode = 'rgb',
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE)

STEP_SIZE_TEST = test_generator.n / test_generator.batch_size 

perf=model.evaluate_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)
pred=model.predict_generator(test_generator,  steps=STEP_SIZE_TEST,  verbose=1)

In [ ]:
print(perf)

In [ ]:
count = 0 
for el in zip(test_generator.filenames,pred): 
    count = count +1 
    #print(el[1]*0.5)
    #print(IOU(el[1],el[1]*0.5))
    image = el[0]
    #print(image.shape)
    classf = el[1]
    #print(classf)
    img_pil = Image.open(image)
    img_pil = img_pil.resize((224,224), Image.ANTIALIAS)
    img1 = ImageDraw.Draw(img_pil)
    img1.rectangle([(classf[0],classf[1]),(classf[0]+classf[2],classf[1]+classf[3])], outline ="red") 
    #img_pil.show()
    img_pil.save("/home/lorenzo.stacchio/ML/Machine_learning_project/dataset/results/output"+ str(count) + ".png")
    #img_test = np.reshape(image, (224,224))
    #print(img.shape)
    #plt.imshow(image.astype(np.uint8))
    #plt.imshow(img_test.astype(np.uint8), cmap='Greys_r',vmin=0, vmax=255)
    #plt.show()
    if count==30:
        break